In [1]:
import tensorflow as tf
import os
from spatial_transformer import transformer
import numpy as np
from tf_utils import weight_variable, bias_variable, dense_to_one_hot
from create_imbalance import create_imbalance
from augment_data import create_augmented
from slim_models import stn_net
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
import matplotlib.cm as cm
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pickle
import create_noisy_datasets

%matplotlib inline

In [2]:
mnist_cluttered = "./data/mnist_cluttered_60x60_6distortions.npz"
data = np.load(mnist_cluttered)
X_train, y_train = data['x_train'], np.argmax(data['y_train'], axis=-1)
X_valid, y_valid = data['x_valid'], np.argmax(data['y_valid'], axis=-1)
X_test, y_test = data['x_test'], np.argmax(data['y_test'], axis=-1)

In [3]:
# X_train_imbalanced, y_train_imbalanced, imbalanced_class_list = create_imbalance(X_train,y_train,X_valid,y_valid)

# X_train_augmented, y_train_augmented = create_augmented(X_train_imbalanced,y_train_imbalanced,imbalanced_class_list)

In [4]:
#pois_noise,gaus_noise,gaus_noise_2, papp_noise = create_noisy_datasets.create_noisy_background(X_train)

In [5]:
Y_train = dense_to_one_hot(y_train, n_classes=10)
Y_valid = dense_to_one_hot(y_valid, n_classes=10)
Y_test = dense_to_one_hot(y_test, n_classes=10)
# Y_train_imbalanced = dense_to_one_hot(y_train_imbalanced, n_classes = 10)
# Y_train_augmented = dense_to_one_hot(y_train_augmented, n_classes = 10)

In [6]:
def draw_bounding_box(ax,location_param,c):
    tx = 30*location_param[2]
    ty = 30*location_param[5]
    a = location_param[0]
    b = location_param[1]
    c = location_param[3]
    d = location_param[4]
    sx = np.sqrt(np.power(a,2)+np.power(b,2))
    sy = np.sqrt(np.power(c,2) + np.power(d,2))
    theta = np.arctan2(c,d)
    w = 60 * sx
    h = 60 * sy
    cmap = cm.jet
    ax.add_patch(
        Rectangle(
            (tx+30-w/2, ty+30-h/2),
            w,
            h,
            angle= theta,
            fill=False,# remove background
            color = 'white'
        )
    )

In [7]:
def plot_transformer_images(X_valid, idxs, sess, label, predictions, h_trans,loc_params, x ,y , keep_prob, locations):
    plt_idx = 1
    f, axarr = plt.subplots(1, len(idxs)*2,figsize=(len(idxs)*40,20))
    for plt_idx,idx in enumerate(idxs):
        X_valid_0 = np.reshape(X_valid[idx],(1,3600))
        Y_valid_0 = np.reshape(Y_valid[idx],(1,10))
        val_label_0, val_predictions_0, h_trans_0,loc_params_0 = sess.run([label, predictions, h_trans,loc_params], feed_dict={x: X_valid_0, y: Y_valid_0, keep_prob: 1.0})
        
        locations.append(loc_params_0)
        if val_label_0 == val_predictions_0:
            label_color = 'black'
        else:
            label_color = 'red'
        axarr[plt_idx*2-1]
        axarr[plt_idx*2-1].set_title("L: {}, P: {}".format( val_label_0, val_predictions_0), {'fontsize': 8},  color=label_color)
        axarr[plt_idx*2-1].xaxis.set_ticks([])
        axarr[plt_idx*2-1].yaxis.set_ticks([])
        #axarr[plt_idx*2-1].yticks([])
        axarr[plt_idx*2-1].imshow(X_valid_0.reshape((60, 60)))
        draw_bounding_box(axarr[plt_idx*2-1],loc_params_0[0],0.5)
        
        axarr[plt_idx*2].xaxis.set_ticks([])
        axarr[plt_idx*2].yaxis.set_ticks([])
        axarr[plt_idx*2].imshow(h_trans_0.reshape((60, 60)))
    plt.show()

In [8]:
def get_batch(X, Y, batch_size):
    indexes = np.random.randint(X.shape[0], size=batch_size)
    return X[indexes,:],Y[indexes,:]

In [9]:
def split_data_into_buckets(X,Y):
    Y_bucket = np.unique(Y,axis=0)
    X_bucket = []
    for label in Y_bucket:
        X_bucket.append(X[(Y == label).all(axis=1)])
    return X_bucket, Y_bucket

In [10]:
def get_batches(X_buckets, Y_buckets, X, Y,batch_size):
    batch_size = int(batch_size/10)
    batch_xa = np.zeros((batch_size*10,X_buckets[0].shape[1]),dtype=np.float32)
    batch_ya = np.zeros((batch_size*10,Y_buckets.shape[1]),dtype=np.float32)
    batch_xp = np.zeros((batch_size*10,X_buckets[0].shape[1]),dtype=np.float32)
    batch_yp = np.zeros((batch_size*10,Y_buckets.shape[1]),dtype=np.float32)
    batch_xn = np.zeros((batch_size*10,X_buckets[0].shape[1]),dtype=np.float32) 
    batch_yn = np.zeros((batch_size*10,Y_buckets.shape[1]),dtype=np.float32)
    for i,x_bucket in enumerate(X_buckets):
        
        y_bucket = np.repeat(Y_buckets[[i]],[x_bucket.shape[0]],axis = 0)
        batch_x, batch_y = get_batch(x_bucket,y_bucket,batch_size)
        batch_xa[batch_size*i:batch_size*(i+1)] = batch_x
        batch_ya[batch_size*i:batch_size*(i+1)] = batch_y
        
        batch_x,batch_y = get_batch(x_bucket,y_bucket,batch_size)
        batch_xp[batch_size*i:batch_size*(i+1)] = batch_x
        batch_yp[batch_size*i:batch_size*(i+1)] = batch_y
        
        batch_x,batch_y = get_batch(X[(Y != Y[0]).any(axis=1)],Y[(Y != Y[0]).any(axis=1)],batch_size)
        batch_xn[batch_size*i:batch_size*(i+1)] = batch_x
        batch_yn[batch_size*i:batch_size*(i+1)] = batch_y
        
    return batch_xa, batch_ya, batch_xp, batch_yp, batch_xn, batch_yn

In [11]:
def print_labels(l_a,l_p,l_n, path):
    # print(np.argmax(np.vstack((l_a,l_p,l_n)),axis=1))
    f = open(path, 'w+')
    list(map(lambda x:print(x,file=f), np.argmax(np.vstack((l_a,l_p,l_n)),axis=1)))
    f.close()

In [12]:
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    
    return spriteimage

def vector_to_matrix_mnist(mnist_digits):
    """Reshapes normal mnist digit (batch,28*28) to matrix (batch,28,28)"""
    return np.reshape(mnist_digits,(-1,60,60))

def invert_grayscale(mnist_digits):
    """ Makes black white, and white black """
    return 1-mnist_digits

In [13]:
def build_spite(batch, path):
    to_visualise = vector_to_matrix_mnist(batch)
    to_visualise = invert_grayscale(to_visualise)

    sprite_image = create_sprite_image(to_visualise)

    plt.imsave(path,sprite_image,cmap='gray')
    # plt.imshow(sprite_image,cmap='gray')
    # plt.show()

In [14]:
log_base = os.path.join(os.path.abspath('.'),'logs','embeddings')
file_writer = tf.summary.FileWriter(log_base)

config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
embedding_config = config.embeddings.add()
embedding_config.tensor_name = "embedding"
embedding_config.sprite.image_path =  os.path.join(log_base,"sprite.png") 
embedding_config.metadata_path = os.path.join(log_base,"labels.tsv")
# Specify the width and height of a single thumbnail.
embedding_config.sprite.single_image_dim.extend([60, 60])

tf.contrib.tensorboard.plugins.projector.visualize_embeddings(file_writer, config)

In [15]:
X_valid_buckets,Y_valid_buckets = split_data_into_buckets(X_valid, Y_valid)
X_test_buckets,Y_test_buckets = split_data_into_buckets(X_test, Y_test)
l_xa, l_ya, l_xp, l_yp, l_xn, l_yn = get_batches(X_valid_buckets, Y_valid_buckets,X_valid,Y_valid, 100)
print_labels(l_ya,l_yp,l_yn, os.path.join(log_base,"labels.tsv"))
build_spite(np.vstack((l_xa, l_xp, l_xn)), os.path.join(log_base,"sprite.png"))

In [16]:
def model(experiment, X,Y,id_,training_set_percentage, batch_size, number_of_iterations, learning_rate, margin,values, log_dir, metrics, confusion_matrices, localization, training_curves):
    
    
    graph = tf.Graph()
    with graph.as_default():
        x_a = tf.placeholder(tf.float32, [batch_size, 3600])
        y_a = tf.placeholder(tf.float32, [batch_size, 10])
        x_p = tf.placeholder(tf.float32, [batch_size, 3600])
        y_p = tf.placeholder(tf.float32, [batch_size, 10])
        x_n = tf.placeholder(tf.float32, [batch_size, 3600])
        y_n = tf.placeholder(tf.float32, [batch_size, 10])
        keep_prob = tf.placeholder(tf.float32)
        optimizer, predictions, label, h_trans, loc_params, summ, training_accuracy, validation_accuracy, input_image, localized_image, embedding = stn_net(x_a, y_a, x_p, y_p, x_n, y_n, keep_prob, learning_rate, margin,values, log_dir, batch_size)

        with tf.Session() as sess:

            file_writer = tf.summary.FileWriter(log_dir)
            sess.run(tf.global_variables_initializer())
            file_writer.add_graph(graph=sess.graph)
            saver = tf.train.Saver()

            locations = []
            biases = []
            train_accs = []
            val_accs = []
            i = 0
            results_fmt = '[{:%H:%M:%S}] Training Set %{} Batch Size {},Accuracy: {:.3f},F1-macro: {:.3f},F1-micro: {:.3f}, Precision: {:.3f},Recall: {:.3f}'
            X_buckets,Y_buckets = split_data_into_buckets(X,Y)
            for index in range(number_of_iterations):
                batch_xa, batch_ya, batch_xp, batch_yp, batch_xn, batch_yn = get_batches(X_buckets, Y_buckets,X,Y, batch_size)

                # Select batches
                sess.run(optimizer, feed_dict={x_a: batch_xa, y_a: batch_ya,
                                               x_p: batch_xp, y_p: batch_yp,
                                               x_n: batch_xn, y_n: batch_yn, keep_prob: 0.8})
                if index % 100 == 0: 
                    batch_xa, batch_ya, batch_xp, batch_yp, batch_xn, batch_yn = get_batches(X_valid_buckets, Y_valid_buckets,X_valid,Y_valid, batch_size)
                    val_label, val_predictions, s,v = sess.run([label, predictions, summ, validation_accuracy], feed_dict={x_a: batch_xa, y_a: batch_ya,
                                               x_p: batch_xp, y_p: batch_yp,
                                               x_n: batch_xn, y_n: batch_yn, keep_prob: 1.0})
                    file_writer.add_summary(s, index)
                    file_writer.add_summary(v, index)
                    val_acc = accuracy_score(val_label, val_predictions)
                    batch_xa, batch_ya, batch_xp, batch_yp, batch_xn, batch_yn = get_batches(X_buckets, Y_buckets,X,Y, batch_size)
                    train_label, train_predictions, t = sess.run([label, predictions, training_accuracy], feed_dict={x_a: batch_xa, y_a: batch_ya,
                                               x_p: batch_xp, y_p: batch_yp,
                                               x_n: batch_xn, y_n: batch_yn, keep_prob: 1.0})
                    file_writer.add_summary(t, index)
                    train_acc = accuracy_score(train_label, train_predictions)

                    train_accs.append(train_acc)
                    val_accs.append(val_acc)
                    
                    
                    i, l, loc_params_0= sess.run([input_image, localized_image, loc_params],feed_dict={x_a: l_xa, y_a: l_ya,
                                               x_p: l_xp, y_p: l_yp,
                                               x_n: l_xn, y_n: l_yn, keep_prob: 1.0})
                    file_writer.add_summary(i, index)
                    file_writer.add_summary(l, index)
                    
                    locations.append(loc_params_0)
                    
                    print('--------> {}/{} Iter, VAL ACC: {:.3f}, TRAIN ACC: {:.3f}'.format(index, number_of_iterations, val_acc, train_acc))

                if index % 2000 ==0:
                    e = sess.run([embedding],feed_dict={x_a: l_xa, y_a: l_ya,
                                               x_p: l_xp, y_p: l_yp,
                                               x_n: l_xn, y_n: l_yn, keep_prob: 1.0})
                    saver.save(sess, os.path.join(log_dir, "saved-model.ckpt"), index)

            batch_xa, batch_ya, batch_xp, batch_yp, batch_xn, batch_yn = get_batches(X_test_buckets, Y_test_buckets,X_test,Y_test, batch_size)
            test_label, test_predictions = sess.run([label, predictions],feed_dict={x_a: batch_xa, y_a: batch_ya,
                                               x_p: batch_xp, y_p: batch_yp,
                                               x_n: batch_xn, y_n: batch_yn, keep_prob: 1.0})
            test_acc = accuracy_score(test_label, test_predictions)
            test_f1_micro = f1_score(test_label, test_predictions, average='micro')
            test_f1_macro = f1_score(test_label, test_predictions, average='macro')
            test_prec = precision_score(test_label, test_predictions, average='micro') 
            test_recall = recall_score(test_label, test_predictions, average='micro')
            test_confusion_matrics = confusion_matrix(test_label, test_predictions)

            #plot_transformer_images(X_valid, np.linspace(0, 999, num=11,dtype=np.int16).tolist(),sess, label, predictions, 
            #                    h_trans, loc_params, x ,y , keep_prob, locations)
            #plt.savefig('localization_figures/e_{}-d_{}-tp_{}-bs_{}'.format(experiment,id_,training_set_percentage, batch_size))
            metrics.append([test_acc,test_f1_macro,test_prec,test_recall])
            training_curves.append([train_accs,val_accs])
            localization.append([locations,biases])
            confusion_matrices.append(test_confusion_matrics)

            print(results_fmt.format(datetime.now(), training_set_percentage, batch_size, 
                                     test_acc, test_f1_macro, test_f1_micro, test_prec,test_recall ))


In [17]:
def make_hparam_string(learning_rate, dataset, batch_size, training_size, strategy ,experiment_number ):
    return "lr_%s-dataset_%s-batch_size_%s-training_size_%s-strategy_%s-ex_%s" % (learning_rate, dataset, batch_size,training_size,strategy,experiment_number)

In [18]:
data_sets = [X_train]#, gaus_noise, gaus_noise_2, papp_noise]
labels = [Y_train]#, Y_train, Y_train, Y_train]
dataset = zip(data_sets,labels)

In [19]:
batch_sizes = [100]#[10,20,50,100,256]
training_set_percentages = [100]#[1, 10, 40 ,80, 100]

In [ ]:
alpha_strategies = [[0.0, 0.25, 0.5, 0.75,1.0],[1.0, 0.75, 0.5, 0.25,0.0],[0.0, 0.0, 0.0, 0.0,0.0],[1.0, 1.0, 1.0, 1.0,1.0]]

In [ ]:
for experiment in range(0,4):
    metrics = []
    localization = []
    confusion_matrices = []
    training_curves = []
    learning_rate = 0.0001
    number_of_iterations = 10000
    
    for id_ in range(len(data_sets)):
        X = data_sets[id_]
        Y = labels[id_]
        for training_set_percentage in training_set_percentages:
            for batch_size in batch_sizes:
                margin = batch_size
                for i,values in enumerate(alpha_strategies):
                    log_dir = os.path.join(log_base,make_hparam_string(learning_rate,id_,batch_size,training_set_percentage,i,experiment))
                    print(log_dir)
                    model(experiment, X,Y,id_,training_set_percentage, batch_size, number_of_iterations, 
                          learning_rate, margin, values,log_dir, metrics, confusion_matrices, localization, training_curves)
    
    pickle.dump( training_curves, open( "metrics/training_curves"+str(experiment)+".pickle", "wb" ) )           
    pickle.dump( localization, open( "metrics/localization"+str(experiment)+".pickle", "wb" ) )           
    pickle.dump( metrics, open( "metrics/metrics"+str(experiment)+".pickle", "wb" ) )
    pickle.dump( confusion_matrices, open( "metrics/confusion_matrices"+str(experiment)+".pickle", "wb" ) )
    

/home/stn2/STN-2/logs/embeddings/lr_0.0001-dataset_0-batch_size_100-training_size_100-strategy_0-ex_0
--------> 0/10000 Iter, VAL ACC: 0.103, TRAIN ACC: 0.120
--------> 100/10000 Iter, VAL ACC: 0.103, TRAIN ACC: 0.100
--------> 200/10000 Iter, VAL ACC: 0.153, TRAIN ACC: 0.147
--------> 300/10000 Iter, VAL ACC: 0.363, TRAIN ACC: 0.387
--------> 400/10000 Iter, VAL ACC: 0.497, TRAIN ACC: 0.487
--------> 500/10000 Iter, VAL ACC: 0.477, TRAIN ACC: 0.523
--------> 600/10000 Iter, VAL ACC: 0.607, TRAIN ACC: 0.620
--------> 700/10000 Iter, VAL ACC: 0.677, TRAIN ACC: 0.620
--------> 800/10000 Iter, VAL ACC: 0.703, TRAIN ACC: 0.690
--------> 900/10000 Iter, VAL ACC: 0.693, TRAIN ACC: 0.753
--------> 1000/10000 Iter, VAL ACC: 0.743, TRAIN ACC: 0.773
--------> 1100/10000 Iter, VAL ACC: 0.760, TRAIN ACC: 0.743
--------> 1200/10000 Iter, VAL ACC: 0.773, TRAIN ACC: 0.783
--------> 1300/10000 Iter, VAL ACC: 0.830, TRAIN ACC: 0.807
--------> 1400/10000 Iter, VAL ACC: 0.793, TRAIN ACC: 0.817
--------> 

--------> 3200/10000 Iter, VAL ACC: 0.550, TRAIN ACC: 0.617
--------> 3300/10000 Iter, VAL ACC: 0.650, TRAIN ACC: 0.610
--------> 3400/10000 Iter, VAL ACC: 0.620, TRAIN ACC: 0.603
--------> 3500/10000 Iter, VAL ACC: 0.637, TRAIN ACC: 0.590
--------> 3600/10000 Iter, VAL ACC: 0.657, TRAIN ACC: 0.610
--------> 3700/10000 Iter, VAL ACC: 0.677, TRAIN ACC: 0.643
--------> 3800/10000 Iter, VAL ACC: 0.657, TRAIN ACC: 0.643
--------> 3900/10000 Iter, VAL ACC: 0.640, TRAIN ACC: 0.697
--------> 4000/10000 Iter, VAL ACC: 0.633, TRAIN ACC: 0.670
--------> 4100/10000 Iter, VAL ACC: 0.663, TRAIN ACC: 0.707
--------> 4200/10000 Iter, VAL ACC: 0.697, TRAIN ACC: 0.733
--------> 4300/10000 Iter, VAL ACC: 0.723, TRAIN ACC: 0.720
--------> 4400/10000 Iter, VAL ACC: 0.697, TRAIN ACC: 0.737
--------> 4500/10000 Iter, VAL ACC: 0.677, TRAIN ACC: 0.750
--------> 4600/10000 Iter, VAL ACC: 0.733, TRAIN ACC: 0.723
--------> 4700/10000 Iter, VAL ACC: 0.717, TRAIN ACC: 0.740
--------> 4800/10000 Iter, VAL ACC: 0.75

--------> 6500/10000 Iter, VAL ACC: 0.880, TRAIN ACC: 0.893
--------> 6600/10000 Iter, VAL ACC: 0.903, TRAIN ACC: 0.907
--------> 6700/10000 Iter, VAL ACC: 0.910, TRAIN ACC: 0.907
--------> 6800/10000 Iter, VAL ACC: 0.893, TRAIN ACC: 0.937
--------> 6900/10000 Iter, VAL ACC: 0.880, TRAIN ACC: 0.910
--------> 7000/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.927
--------> 7100/10000 Iter, VAL ACC: 0.913, TRAIN ACC: 0.910
--------> 7200/10000 Iter, VAL ACC: 0.893, TRAIN ACC: 0.937
--------> 7300/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.920
--------> 7400/10000 Iter, VAL ACC: 0.893, TRAIN ACC: 0.910
--------> 7500/10000 Iter, VAL ACC: 0.863, TRAIN ACC: 0.940
--------> 7600/10000 Iter, VAL ACC: 0.903, TRAIN ACC: 0.953
--------> 7700/10000 Iter, VAL ACC: 0.903, TRAIN ACC: 0.917
--------> 7800/10000 Iter, VAL ACC: 0.927, TRAIN ACC: 0.903
--------> 7900/10000 Iter, VAL ACC: 0.917, TRAIN ACC: 0.897
--------> 8000/10000 Iter, VAL ACC: 0.910, TRAIN ACC: 0.907
--------> 8100/10000 Iter, VAL ACC: 0.91

--------> 9800/10000 Iter, VAL ACC: 0.060, TRAIN ACC: 0.083
--------> 9900/10000 Iter, VAL ACC: 0.080, TRAIN ACC: 0.073


/home/stn2/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

--------> 2600/10000 Iter, VAL ACC: 0.470, TRAIN ACC: 0.497
--------> 2700/10000 Iter, VAL ACC: 0.430, TRAIN ACC: 0.477
--------> 2800/10000 Iter, VAL ACC: 0.427, TRAIN ACC: 0.500
--------> 2900/10000 Iter, VAL ACC: 0.417, TRAIN ACC: 0.490
--------> 3000/10000 Iter, VAL ACC: 0.500, TRAIN ACC: 0.540
--------> 3100/10000 Iter, VAL ACC: 0.517, TRAIN ACC: 0.537
--------> 3200/10000 Iter, VAL ACC: 0.497, TRAIN ACC: 0.563
--------> 3300/10000 Iter, VAL ACC: 0.567, TRAIN ACC: 0.520
--------> 3400/10000 Iter, VAL ACC: 0.527, TRAIN ACC: 0.517
--------> 3500/10000 Iter, VAL ACC: 0.517, TRAIN ACC: 0.603
--------> 3600/10000 Iter, VAL ACC: 0.520, TRAIN ACC: 0.600
--------> 3700/10000 Iter, VAL ACC: 0.597, TRAIN ACC: 0.597
--------> 3800/10000 Iter, VAL ACC: 0.530, TRAIN ACC: 0.603
--------> 3900/10000 Iter, VAL ACC: 0.627, TRAIN ACC: 0.580
--------> 4000/10000 Iter, VAL ACC: 0.563, TRAIN ACC: 0.637
--------> 4100/10000 Iter, VAL ACC: 0.627, TRAIN ACC: 0.647
--------> 4200/10000 Iter, VAL ACC: 0.64

--------> 5900/10000 Iter, VAL ACC: 0.900, TRAIN ACC: 0.923
--------> 6000/10000 Iter, VAL ACC: 0.900, TRAIN ACC: 0.917
--------> 6100/10000 Iter, VAL ACC: 0.863, TRAIN ACC: 0.927
--------> 6200/10000 Iter, VAL ACC: 0.897, TRAIN ACC: 0.937
--------> 6300/10000 Iter, VAL ACC: 0.887, TRAIN ACC: 0.920
--------> 6400/10000 Iter, VAL ACC: 0.897, TRAIN ACC: 0.910
--------> 6500/10000 Iter, VAL ACC: 0.890, TRAIN ACC: 0.920
--------> 6600/10000 Iter, VAL ACC: 0.877, TRAIN ACC: 0.900
--------> 6700/10000 Iter, VAL ACC: 0.930, TRAIN ACC: 0.927
--------> 6800/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.927
--------> 6900/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.923
--------> 7000/10000 Iter, VAL ACC: 0.903, TRAIN ACC: 0.917
--------> 7100/10000 Iter, VAL ACC: 0.897, TRAIN ACC: 0.903
--------> 7200/10000 Iter, VAL ACC: 0.923, TRAIN ACC: 0.943
--------> 7300/10000 Iter, VAL ACC: 0.903, TRAIN ACC: 0.910
--------> 7400/10000 Iter, VAL ACC: 0.913, TRAIN ACC: 0.933
--------> 7500/10000 Iter, VAL ACC: 0.91

--------> 9200/10000 Iter, VAL ACC: 0.137, TRAIN ACC: 0.133
--------> 9300/10000 Iter, VAL ACC: 0.120, TRAIN ACC: 0.150
--------> 9400/10000 Iter, VAL ACC: 0.143, TRAIN ACC: 0.100
--------> 9500/10000 Iter, VAL ACC: 0.140, TRAIN ACC: 0.117
--------> 9600/10000 Iter, VAL ACC: 0.147, TRAIN ACC: 0.137
--------> 9700/10000 Iter, VAL ACC: 0.127, TRAIN ACC: 0.120
--------> 9800/10000 Iter, VAL ACC: 0.117, TRAIN ACC: 0.113
--------> 9900/10000 Iter, VAL ACC: 0.150, TRAIN ACC: 0.117
[19:06:44] Training Set %100 Batch Size 100,Accuracy: 0.153,F1-macro: 0.058,F1-micro: 0.153, Precision: 0.153,Recall: 0.153
/home/stn2/STN-2/logs/embeddings/lr_0.0001-dataset_0-batch_size_100-training_size_100-strategy_0-ex_3
--------> 0/10000 Iter, VAL ACC: 0.100, TRAIN ACC: 0.097
--------> 100/10000 Iter, VAL ACC: 0.123, TRAIN ACC: 0.117
--------> 200/10000 Iter, VAL ACC: 0.207, TRAIN ACC: 0.190
--------> 300/10000 Iter, VAL ACC: 0.307, TRAIN ACC: 0.350
--------> 400/10000 Iter, VAL ACC: 0.413, TRAIN ACC: 0.453
-

--------> 2200/10000 Iter, VAL ACC: 0.377, TRAIN ACC: 0.353
--------> 2300/10000 Iter, VAL ACC: 0.380, TRAIN ACC: 0.383
--------> 2400/10000 Iter, VAL ACC: 0.373, TRAIN ACC: 0.453
--------> 2500/10000 Iter, VAL ACC: 0.403, TRAIN ACC: 0.510
--------> 2600/10000 Iter, VAL ACC: 0.437, TRAIN ACC: 0.460
--------> 2700/10000 Iter, VAL ACC: 0.427, TRAIN ACC: 0.443
--------> 2800/10000 Iter, VAL ACC: 0.457, TRAIN ACC: 0.490
--------> 2900/10000 Iter, VAL ACC: 0.477, TRAIN ACC: 0.470
--------> 3000/10000 Iter, VAL ACC: 0.510, TRAIN ACC: 0.550
--------> 3100/10000 Iter, VAL ACC: 0.560, TRAIN ACC: 0.550
--------> 3200/10000 Iter, VAL ACC: 0.530, TRAIN ACC: 0.530
--------> 3300/10000 Iter, VAL ACC: 0.533, TRAIN ACC: 0.587
--------> 3400/10000 Iter, VAL ACC: 0.523, TRAIN ACC: 0.530
--------> 3500/10000 Iter, VAL ACC: 0.517, TRAIN ACC: 0.593
--------> 3600/10000 Iter, VAL ACC: 0.553, TRAIN ACC: 0.610
--------> 3700/10000 Iter, VAL ACC: 0.613, TRAIN ACC: 0.590
--------> 3800/10000 Iter, VAL ACC: 0.61

--------> 5500/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.897
--------> 5600/10000 Iter, VAL ACC: 0.910, TRAIN ACC: 0.913
--------> 5700/10000 Iter, VAL ACC: 0.877, TRAIN ACC: 0.927
--------> 5800/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.910
--------> 5900/10000 Iter, VAL ACC: 0.880, TRAIN ACC: 0.903
--------> 6000/10000 Iter, VAL ACC: 0.923, TRAIN ACC: 0.907
--------> 6100/10000 Iter, VAL ACC: 0.887, TRAIN ACC: 0.893
--------> 6200/10000 Iter, VAL ACC: 0.887, TRAIN ACC: 0.913
--------> 6300/10000 Iter, VAL ACC: 0.907, TRAIN ACC: 0.933
--------> 6400/10000 Iter, VAL ACC: 0.883, TRAIN ACC: 0.917
--------> 6500/10000 Iter, VAL ACC: 0.887, TRAIN ACC: 0.940
--------> 6600/10000 Iter, VAL ACC: 0.880, TRAIN ACC: 0.930
--------> 6700/10000 Iter, VAL ACC: 0.913, TRAIN ACC: 0.903
--------> 6800/10000 Iter, VAL ACC: 0.870, TRAIN ACC: 0.927
--------> 6900/10000 Iter, VAL ACC: 0.873, TRAIN ACC: 0.930
--------> 7000/10000 Iter, VAL ACC: 0.880, TRAIN ACC: 0.907
--------> 7100/10000 Iter, VAL ACC: 0.90

In [ ]:
# metrics = pickle.load( open( "metrics/metrics0.pickle", "rb" ) )